In [1]:
import pandas as pd
import geopandas as gpd
import gtfs_functions as gtfs
import numpy as np
import pandas as pd,dateparser

from tqdm import tqdm

In [2]:
df = pd.read_excel('../Varanasi Data/Transit Data/Route data/ALL ROUTE OK 24.01.2022.xlsx',sheet_name=['MZD-BabatpurAirport 101'],header=1).dropna(axis=1, how='all')

AttributeError: 'dict' object has no attribute 'dropna'

In [86]:
dfSuperSet = pd.read_excel('../Varanasi Data/Transit Data/Route data/ALL ROUTE OK 24.01.2022.xlsx',sheet_name=None,header=1)

In [65]:
routes=dfSuperSet['50 Bus KM (2)']

In [66]:
routes=routes.drop(columns=['S.No'])

In [67]:
routes=routes.rename(columns={'Route No.':'route_id','Route Name':'route_long_name'})
pd.options.display.max_colwidth = 200
routes=routes.loc[0:routes.shape[0]-2]

In [68]:
routes['agency_id']=1

In [69]:
routes

,route_long_name,route_id,Bus,Route Leght,Trip,Total Trip,E-Station KM\n(UP &Down),No Shift,Total KM\n(UP &Down),Total Shift,Total KM,agency_id
0,Mirzamurad-Mohansarai-Cantt-LBS Airport- Babatpur Chauraha,E101,4,53,4,16,106,2,212,8,848,1
1,Mirzamurad-Mohansarai-Cantt-Babatpur Chauraha,E102,6,49,4,24,98,2,196,12,1176,1
2,Mirzamurad-Kachhawan-Mohansarai- Cantt- Bhelupur-Lanka,E103,3,48,4,12,96,2,192,6,576,1
3,Mirzamurad-Mohansarai-Cantt- Bhojubeer-Sindhora,E104,7,54,4,28,108,2,216,14,1512,1
4,Mirzamurad-Mohansarai-Cantt-Pandaypur-Dharsauna-Danganj,E105,5,53,4,20,106,2,212,10,1060,1
5,Mirzamurad-Mohansarai-Cantt-Pandeypur-Chubeypur-Markandey Mahadev,E106,6,57,4,24,114,2,228,12,1368,1
6,Mirzamurad-Mohansarai-Cantt-Ashapur-Sarnath -Lanka,E107,3,45,8,24,112,2,224,6,672,1
7,Mirzamurad-Mohansarai-Cantt-Ashapur-Lanka -Sarnath,NaN,3,45,8,24,106,2,212,6,636,1
8,Mirzamurad- Akhari-DLW-Cantt-JHV Mall- Kachahari- Ashapur- Sarnath- Cantt-Bhelupur- Lanka-DLW- Cantt,E108,2,54,4,8,106,2,212,4,424,1
9,Mirzamurad-Mohansarai-Cantt-Rajghat-Cantt-DLW-Lanka-Bhelupur-Cantt,E109,3,42,8,24,111,2,222,6,666,1


In [70]:
routes['route_id'].mask((routes.route_id.shift(-1).isnull())&(routes.route_id.index!=routes.shape[0]-1), routes['route_id']+'A',inplace=True)
routes['route_id'].mask(routes.route_id.isnull(),routes['route_id'].shift().str[:-1],inplace=True)

In [74]:
routes[['route_color', 'route_text_color']]=None
routes['route_short_name']=list(dfSuperSet.keys())[1:]
routes['route_short_name']=routes['route_short_name'].str.strip()
routes['route_long_name']=routes['route_long_name'].str.strip()

/tmp/ipykernel_89802/763008983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  routes[['route_color', 'route_text_color']]=None
/tmp/ipykernel_89802/763008983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  routes['route_short_name']=list(dfSuperSet.keys())[1:]
/tmp/ipykernel_89802/763008983.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [75]:

routes=routes[['route_id','agency_id','route_short_name','route_long_name','route_color', 'route_text_color']]


In [76]:
routes

,route_id,agency_id,route_short_name,route_long_name,route_color,route_text_color
0,E101,1,MZD-BabatpurAirport 101,Mirzamurad-Mohansarai-Cantt-LBS Airport- Babatpur Chauraha,None,None
1,E102,1,Mirzamurad-Babatpur 102,Mirzamurad-Mohansarai-Cantt-Babatpur Chauraha,None,None
2,E103,1,Mirzamurad-Cantt-Lanka103,Mirzamurad-Kachhawan-Mohansarai- Cantt- Bhelupur-Lanka,None,None
3,E104,1,Mirzamurad-Sindharo104,Mirzamurad-Mohansarai-Cantt- Bhojubeer-Sindhora,None,None
4,E105,1,Mirzamurad-Darsauna-Danganj 105,Mirzamurad-Mohansarai-Cantt-Pandaypur-Dharsauna-Danganj,None,None
5,E106,1,Mirzamurad-Chubpur-MARKAND 106,Mirzamurad-Mohansarai-Cantt-Pandeypur-Chubeypur-Markandey Mahadev,None,None
6,E107A,1,Mirzamurad-Sarnath-Lanka 107A,Mirzamurad-Mohansarai-Cantt-Ashapur-Sarnath -Lanka,None,None
7,E107,1,MZD-Lanka-Sarnath 107,Mirzamurad-Mohansarai-Cantt-Ashapur-Lanka -Sarnath,None,None
8,E108,1,MZD-DLW-Cantt-Sarnth-Lanka108,Mirzamurad- Akhari-DLW-Cantt-JHV Mall- Kachahari- Ashapur- Sarnath- Cantt-Bhelupur- Lanka-DLW- Cantt,None,None
9,E109,1,MZD-Cantt-Rajgt- DLW-Lanka109,Mirzamurad-Mohansarai-Cantt-Rajghat-Cantt-DLW-Lanka-Bhelupur-Cantt,None,None


In [78]:
routes.to_csv(r'../GTFS/routes.txt',index=False)

In [79]:
#Reads through all excel sheets and extract all stops with names and coordinates
stops=pd.DataFrame()
for route in dfSuperSet.keys():
    #skips the first sheet of excel file
    if route=='50 Bus KM (2)':
        pass
    else:
        c=pd.DataFrame(dfSuperSet[route])
        print(c.columns[0])
        #Change route name column to stops
        c=c.rename(columns={c.columns[0]:'stop_name'})
        #Checks for column that has substring Latitu in it
        latlon=[col for col in c.columns if 'Latitu' in col]
        print(route)
        print(latlon)
        
        c=c[['stop_name',latlon[0]]]
        #Check for null rows that separates different directions of routes
        index_list=c[c.isnull().all(axis=1)].index
    
        stops=stops.append(c.iloc[1:index_list[0]])
        
        for i in range(len(index_list)):
            if i!= len(index_list)-1:
                stops=stops.append(c.iloc[c[c.isnull().all(axis=1)].index[i]+4:c[c.isnull().all(axis=1)].index[i+1]])
        
    
    
    

Route NO.
(E101)
MZD-BabatpurAirport 101
['Latitude\nLongitude']
Route NO.
(E102)
Mirzamurad-Babatpur 102
['Latitude\nLongitude']
Route NO.
(E103)
Mirzamurad-Cantt-Lanka103
['Latitude\nLongitude']
Route NO.
(E104)
Mirzamurad-Sindharo104
['Latitude\nLongitude']
Route No.
(E105)
Mirzamurad-Darsauna-Danganj 105
['Latitude\nLongitude']
Route No.
(E106)
Mirzamurad-Chubpur-MARKAND 106
['Latitude\nLongitude']
Route No.
(E107)
Mirzamurad-Sarnath-Lanka 107A
['Latitude\nLongitude']
Route No.
(E107)
MZD-Lanka-Sarnath 107 
['Latitude\nLongitude']
Route No.
(E108)
MZD-DLW-Cantt-Sarnth-Lanka108
['Latitude\nLongitude']
Route No.
(E109)
MZD-Cantt-Rajgt- DLW-Lanka109
['Latitude\nLongitude']
Route No.
(E110)
MZD-CANT-GODAULIA-CANTT110
['Latitude\nLongitude']
Route No.
(E111)
MZD-CANT-GODAULIA-CANTT111
['Latitude\nLongitude']


/tmp/ipykernel_89802/2724134904.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stops=stops.append(c.iloc[1:index_list[0]])
/tmp/ipykernel_89802/2724134904.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stops=stops.append(c.iloc[c[c.isnull().all(axis=1)].index[i]+4:c[c.isnull().all(axis=1)].index[i+1]])
/tmp/ipykernel_89802/2724134904.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stops=stops.append(c.iloc[1:index_list[0]])
/tmp/ipykernel_89802/2724134904.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stops=stops.append(c.iloc[c[c.isnull().all(axis=1)].index[i]+4:c[c.isnull().all(axis=1)].index[i+1]])
/tmp/ipykern

In [80]:
stops=stops.drop_duplicates().reset_index(drop=True)

In [81]:
stops['stop_id']=stops.index
stops[['stop_lat','stop_lon']] = stops['Latitude\nLongitude'].str.split(",",expand=True)
stops[['stop_lat','stop_lon']]=stops[['stop_lat','stop_lon']].astype(float)
stops.drop(columns=['Latitude\nLongitude'])

,stop_name,stop_id,stop_lat,stop_lon
0,SADHU KI KUTIYA,0,25.279853,82.799645
1,RAKHAUNA,1,25.271908,82.821388
2,RAJATALAB,2,25.269400,82.846250
3,MOHANSARAI,3,25.274509,82.871995
4,SAHABABAD,4,25.277839,82.896804
...,...,...,...,...
231,TALIYABHAG,231,25.327688,82.995952
232,CHAUKAGHAT,232,25.332438,82.996325
233,KABIR CHAURA,233,25.319730,83.009175
234,MAIDAGIN,234,25.318957,83.013142


In [82]:
stops[['stop_code','stop_desc','zone_id','stop_url','location_type','parent_station']]=None
stops=stops[['stop_id','stop_code','stop_name','stop_desc','stop_lat','stop_lon','zone_id','stop_url','location_type','parent_station']]

In [83]:
stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,0,None,SADHU KI KUTIYA,None,25.279853,82.799645,None,None,None,None
1,1,None,RAKHAUNA,None,25.271908,82.821388,None,None,None,None
2,2,None,RAJATALAB,None,25.269400,82.846250,None,None,None,None
3,3,None,MOHANSARAI,None,25.274509,82.871995,None,None,None,None
4,4,None,SAHABABAD,None,25.277839,82.896804,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
231,231,None,TALIYABHAG,None,25.327688,82.995952,None,None,None,None
232,232,None,CHAUKAGHAT,None,25.332438,82.996325,None,None,None,None
233,233,None,KABIR CHAURA,None,25.319730,83.009175,None,None,None,None
234,234,None,MAIDAGIN,None,25.318957,83.013142,None,None,None,None


In [149]:
stops.to_csv(r'../GTFS/stops.txt',index=False)

In [224]:
#Reads through all excel sheets and extract all stops with names and coordinates
trips=pd.DataFrame(columns=['trip_id','route_id','service_id','trip_headsign','direction_id','block_id','shape_id'])
for route in dfSuperSet.keys():
    #skips the first sheet of excel file
    if route=='50 Bus KM (2)':
        pass
    else:
        #Reads the whole sheet as one dataframe
        c=pd.DataFrame(dfSuperSet[route]).iloc[:-1].dropna(axis=1,how='all')
        #stores the trips from columns into a list
        tripcount=[col for col in c.columns if 'Trip' in col]
    
        index_list=c[c.isnull().all(axis=1)].index
        DirectionCount=0
        #iterates through each table of the sheet
        for i in range(len(index_list)):
            if index_list[i]==index_list[0]:
                temp=c.iloc[0:index_list[i]]
                
                for count in range(len(tripcount)):
                        loc=trips.shape[0]
                        trips.loc[loc,'block_id']=temp.loc[0,tripcount[count]]
                        trips.loc[loc,'direction_id']= "Direction " + str(DirectionCount)
                DirectionCount= DirectionCount+1
             
            #skips consecitive empty lines   
            elif c.iloc[index_list[i-1]+1:index_list[i]].empty:
                print(i)
                print('consective nan')
                pass
            else:
                temp=c.iloc[index_list[i-1]+1:index_list[i]].reset_index(drop=True)
                
                if len(temp)!=1:
                    for count in range(len(tripcount)):
                        loc=trips.shape[0]
                        trips.loc[loc,'block_id']=temp.loc[2,tripcount[count]]
                        trips.loc[loc,'direction_id']= "Direction " + str(DirectionCount)
                DirectionCount= DirectionCount+1
            print(route)
            #
            trips.route_id.mask(trips.route_id.isnull(),routes[routes['route_short_name']==route.strip()].reset_index(drop=True).route_id[0],inplace=True)   

MZD-BabatpurAirport 101
MZD-BabatpurAirport 101
Mirzamurad-Babatpur 102
Mirzamurad-Babatpur 102
Mirzamurad-Cantt-Lanka103
Mirzamurad-Cantt-Lanka103
Mirzamurad-Sindharo104
Mirzamurad-Sindharo104
Mirzamurad-Darsauna-Danganj 105
Mirzamurad-Darsauna-Danganj 105
Mirzamurad-Chubpur-MARKAND 106
1
consective nan
Mirzamurad-Chubpur-MARKAND 106
Mirzamurad-Chubpur-MARKAND 106
Mirzamurad-Chubpur-MARKAND 106
4
consective nan
Mirzamurad-Chubpur-MARKAND 106
5
consective nan
Mirzamurad-Chubpur-MARKAND 106
6
consective nan
Mirzamurad-Chubpur-MARKAND 106
7
consective nan
Mirzamurad-Chubpur-MARKAND 106
8
consective nan
Mirzamurad-Chubpur-MARKAND 106
Mirzamurad-Sarnath-Lanka 107A
Mirzamurad-Sarnath-Lanka 107A
Mirzamurad-Sarnath-Lanka 107A
MZD-Lanka-Sarnath 107 
MZD-Lanka-Sarnath 107 
MZD-Lanka-Sarnath 107 
MZD-DLW-Cantt-Sarnth-Lanka108
MZD-DLW-Cantt-Sarnth-Lanka108
MZD-DLW-Cantt-Sarnth-Lanka108
MZD-Cantt-Rajgt- DLW-Lanka109
MZD-Cantt-Rajgt- DLW-Lanka109
MZD-Cantt-Rajgt- DLW-Lanka109
MZD-CANT-GODAULIA-CANT

In [225]:
trips=trips[trips.block_id.notnull()].reset_index(drop=True)

In [236]:
trips['trip_id']=trips.index
trips['service_id']=1
trips['block_id']=trips['block_id'].str.replace('\n ','')
trips['block_id']=trips['block_id'].str.replace('\n','')
trips['block_id']=trips['block_id'].str.replace(' ','')


In [237]:
trips

,trip_id,route_id,service_id,trip_headsign,direction_id,block_id,shape_id
0,0,E101,1,NaN,Direction 0,UP65KT5041,NaN
1,1,E101,1,NaN,Direction 0,UP32LT1406,NaN
2,2,E101,1,NaN,Direction 0,UP65LT1409,NaN
3,3,E101,1,NaN,Direction 0,UP32PN6056,NaN
4,4,E101,1,NaN,Direction 0,UP65KT5041,NaN
...,...,...,...,...,...,...,...
257,257,E111,1,NaN,Direction 2,UP65LT1395,NaN
258,258,E111,1,NaN,Direction 2,UP32PN8724,NaN
259,259,E111,1,NaN,Direction 2,UP32KT5047,NaN
260,260,E111,1,NaN,Direction 2,UP65KT5042,NaN


In [238]:
trips.to_csv(r'../GTFS/trips.txt',index=False)

In [ ]:
#Reads through all excel sheets and extract all stops with names and coordinates
stop_times=pd.DataFrame(columns=['trip_id',	'arrival_time',	'departure_time',	'stop_id',	'stop_sequence',	'stop_head_sign',	'shape_dist_traveled',	'halt',	'Stop_name'])
for route in dfSuperSet.keys():
    #skips the first sheet of excel file
    if route=='50 Bus KM (2)':
        pass
    else:
        #Reads the whole sheet as one dataframe
        c=pd.DataFrame(dfSuperSet[route]).iloc[:-1].dropna(axis=1,how='all')
        #stores the trips from columns into a list
        tripcount=[col for col in c.columns if 'Trip' in col]
    
        index_list=c[c.isnull().all(axis=1)].index
        DirectionCount=0
        #iterates through each table of the sheet
        for i in range(len(index_list)):
            if index_list[i]==index_list[0]:
                temp=c.iloc[0:index_list[i]]
                
                for count in range(len(tripcount)):
                        loc=trips.shape[0]
                        trips.loc[loc,'block_id']=temp.loc[0,tripcount[count]]
                        trips.loc[loc,'direction_id']= "Direction " + str(DirectionCount)
                DirectionCount= DirectionCount+1
             
            #skips consecitive empty lines   
            elif c.iloc[index_list[i-1]+1:index_list[i]].empty:
                print(i)
                print('consective nan')
                pass
            else:
                temp=c.iloc[index_list[i-1]+1:index_list[i]].reset_index(drop=True)
                
                if len(temp)!=1:
                    for count in range(len(tripcount)):
                        loc=trips.shape[0]
                        trips.loc[loc,'block_id']=temp.loc[2,tripcount[count]]
                        trips.loc[loc,'direction_id']= "Direction " + str(DirectionCount)
                DirectionCount= DirectionCount+1
            print(route)
            #
            trips.route_id.mask(trips.route_id.isnull(),routes[routes['route_short_name']==route.strip()].reset_index(drop=True).route_id[0],inplace=True)   

In [249]:
stop_times=pd.DataFrame(columns=['trip_id',	'arrival_time',	'departure_time',	'stop_id',	'stop_sequence',	'stop_head_sign',	'shape_dist_traveled',	'halt',	'stop_name'])
trip_id=0
#Reads the whole sheet as one dataframe
c=pd.DataFrame(dfSuperSet['Mirzamurad-Sindharo104']).iloc[:-1].dropna(axis=1,how='all')

#stores the trips from columns into a list
tripcount=[col for col in c.columns if 'Trip' in col]
total_km=[col for col in c.columns if 'Km' in col][0]
index_list=c[c.isnull().all(axis=1)].index

#iterates through each table of the sheet
for i in range(len(index_list)):
    if index_list[i]==index_list[0]:
        temp=c.iloc[1:index_list[i]]
        temp=temp.rename(columns={c.columns[0]:'stop_name'})
        countrows=temp.shape[0]
        temp
        for count in range(len(tripcount)):
                # stop_times[['stop_name','departure_time','halt','shape_dist_traveled']]=stop_times[['stop_name','departure_time','halt','shape_dist_traveled']].concat(temp['stop_name',tripcount[count],'HLT',total_km])
                stop_times=stop_times.append({'stop_name':temp['stop_name']},ignore_index=True)
                stop_times.trip_id.mask(stop_times.trip_id.isnull(),trip_id,inplace=True)
                trip_id=trip_id+1
                
                
        
        
    #skips consecitive empty lines   
    elif c.iloc[index_list[i-1]+1:index_list[i]].empty:
        print(i)
        print('consective nan')
        pass
    else:
        pass
        # temp=c.iloc[index_list[i-1]+1:index_list[i]].reset_index(drop=True)
        
        # if len(temp)!=1:
        #     for count in range(len(tripcount)):
        #         loc=trips.shape[0]
        #         trips.loc[loc,'block_id']=temp.loc[2,tripcount[count]]
        #         trips.loc[loc,'direction_id']= "Direction " + str(DirectionCount)
        # DirectionCount= DirectionCount+1
    print(route)
    #
    # trips.route_id.mask(trips.route_id.isnull(),routes[routes['route_short_name']==route.strip()].reset_index(drop=True).route_id[0],inplace=True)   

MZD-CANT-GODAULIA-CANTT111
MZD-CANT-GODAULIA-CANTT111


/tmp/ipykernel_89802/3229616708.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop_name']},ignore_index=True)
/tmp/ipykernel_89802/3229616708.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop_name']},ignore_index=True)
/tmp/ipykernel_89802/3229616708.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop_name']},ignore_index=True)
/tmp/ipykernel_89802/3229616708.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop_name']},ign

In [253]:
c
temp=c.iloc[1:index_list[0]]
temp=temp.rename(columns={c.columns[0]:'stop_name'})

In [260]:
list1=temp['stop_name'].to_list()

In [261]:
list1

['MIRZAMURAD E STATION',
 'MIRZAMURAD BAZAR',
 'KIT',
 'SADHU KI KUTIYA',
 'RAKHAUNA',
 'RAJATALAB',
 'MOHANSARAI',
 'SAHABABAD',
 'JAGATPUR',
 'ROHANIA',
 'BHULLANPUR',
 'MODHAILA',
 'CHANDPUR CHAURAHA',
 'BOULIA',
 'LAHARTARA',
 'CANTT RLY STATION',
 'NADESAR',
 'KACHAHARI',
 'CIRCUIT HOUSE',
 'BHOJUBEER',
 'NATINIYA DAI',
 'LALPUR   T.F.C.',
 'CHANDMARI',
 'BELWARIYA',
 'MURDAHA',
 'AYAR',
 'AHIRAULI',
 'GOSAIPUR',
 'PALAHI PATTI',
 'GARTHAMA',
 'SARAIYA',
 'MARUEE',
 'SINDHORA']

In [239]:
c=pd.DataFrame(dfSuperSet['Mirzamurad-Sindharo104']).iloc[:-1].dropna(axis=1,how='all')
c

,Route NO.\n(E104),KM,Total Km,Time,Total Km.1,HLT,First Trip Bus-1,First Trip Bus-2,First Trip Bus-3,First Trip Bus-4,...,First Trip Bus-6,First Trip Bus-7,Second Trip \nBus-1,Second Trip \nBus-2,Second Trip \nBus-3,Second Trip \nBus-4,Second Trip \nBus-5,Second Trip \nBus-6,Second Trip \nBus-7,Latitude\nLongitude
0,NaN,NaN,NaN,NaN,NaN,NaN,UP65PN\n 8725,UP65LT\n1408,UP65LT\n1405,UP65PN\n5822,...,UP65PN\n 5826,UP32LT\n1797,UP65PN\n 8725,UP65LT\n1408,UP65LT\n1405,UP65PN\n5822,UP65LT \n1390,UP65PN\n 5826,UP32LT\n1797,NaN
1,MIRZAMURAD E STATION,0,0,0,0,0,05:15:00,05:45:00,06:45:00,07:45:00,...,09:15:00,10:15:00,10:45:00,11:15:00,12:15:00,13:15:00,13:45:00,14:45:00,15:45:00,"25.284666, 82.780556"
2,MIRZAMURAD BAZAR,1,1,2.4,10,5,05:22:00,05:52:00,06:52:00,07:52:00,...,09:22:00,10:22:00,10:52:00,11:22:00,12:22:00,13:22:00,13:52:00,14:52:00,15:52:00,"25.285235, 82.776484"
3,KIT,2,3,4.8,10,0,05:26:00,05:56:00,06:56:00,07:56:00,...,09:26:00,10:26:00,10:56:00,11:26:00,12:26:00,13:26:00,13:56:00,14:56:00,15:56:00,"25.282433, 82.791429"
4,SADHU KI KUTIYA,1,4,2.4,15,0,05:28:00,05:58:00,06:58:00,07:58:00,...,09:28:00,10:28:00,10:58:00,11:28:00,12:28:00,13:28:00,13:58:00,14:58:00,15:58:00,"25.279853, 82.799645"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,SADHU KI KUTIYA,2,50,4.8,60,0,09:17:00,09:47:00,10:47:00,11:47:00,...,13:17:00,14:17:00,14:47:00,15:17:00,16:17:00,17:17:00,17:47:00,18:47:00,19:47:00,"25.279515, 82.799523"
68,KIT,1,51,2.4,60,0,09:19:00,09:49:00,10:49:00,11:49:00,...,13:19:00,14:19:00,14:49:00,15:19:00,16:19:00,17:19:00,17:49:00,18:49:00,19:49:00,"25.282121, 82.791342"
69,MIRZAMURAD BAZAR,2,53,4.8,60,0,09:23:00,09:53:00,10:53:00,11:53:00,...,13:23:00,14:23:00,14:53:00,15:23:00,16:23:00,17:23:00,17:53:00,18:53:00,19:53:00,"25.285235, 82.776484"
70,MIRZAMURAD E STATION,1,54,2.4,60,0,09:25:00,09:55:00,10:55:00,11:55:00,...,13:25:00,14:25:00,14:55:00,15:25:00,16:25:00,17:25:00,17:55:00,18:55:00,19:55:00,"25.284666, 82.780556"
